ERA5

- mean, median, std, min, max for temperature values by year, year-month and year-month-day

- maps for both results


## Old Way

### Yearly

In [ ]:
import xarray as xr
import pandas as pd
import os
import numpy as np

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\Temperature\2conversion_year_1979_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store yearly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    # Open the dataset using memory-efficient lazy loading
    ds = xr.open_dataset(file_path, chunks={'valid_time': 500})  # Process in chunks
    
    # Extract the year from the 'valid_time' dimension
    year = ds['valid_time'].dt.year.values[0]  # Extract the year from the first time step
    
    # Get the temperature data as a numpy array
    temp_data = ds['t2m'].values  # Numpy array (all time steps)

    # Compute statistics using numpy's nan functions
    mean = np.nanmean(temp_data, axis=0)
    median = np.nanmedian(temp_data, axis=0)
    std = np.nanstd(temp_data, axis=0)
    max_ = np.nanmax(temp_data, axis=0)
    min_ = np.nanmin(temp_data, axis=0)
    
    # Combine results into a new dataset
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

    # Convert to DataFrame and restructure
    stats_df = stats.to_dataframe().reset_index()
    stats_df['Year'] = year  # Add the year directly from valid_time
    stats_df = stats_df.set_index(['Year', 'latitude', 'longitude'])
    
    # Append to list
    df_list.append(stats_df)

# Concatenate all years into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)


In [ ]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"D:\IPMA\ERA5\Temperature\yearly_temperature_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os

# File path
file_path = r"E:\IPMA\ERA5\Temperature\yearly_temperature_stats.nc"
output_folder = r"E:\IPMA\ERA5\Temperature\maps_year"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Temperature (°C)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'coolwarm'

# Projection (assume data is in PlateCarree, common for lat/lon grids)
data_crs = ccrs.PlateCarree()

# Loop through years
for year in ds['Year'].values:
    fig, ax = plt.subplots(figsize=(8, 6),
                           subplot_kw={'projection': ccrs.PlateCarree()})

    # Select data for current year
    data = ds[var].sel(Year=year)

    # Plot the data
    im = ax.pcolormesh(ds['longitude'], ds['latitude'], data,
                       cmap=cmap, vmin=vmin, vmax=vmax,
                       shading='auto', transform=data_crs)

    # Add features (land, ocean, borders)
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.set_extent([-12, 36, 34, 66], crs=data_crs)  # Adjust to the working area

    # Gridlines (bottom and left only)
    gl = ax.gridlines(draw_labels=True, linewidth=0)
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = True

    # Title and colorbar
    ax.set_title(f'{title} - {year}', fontsize=12)

    # Create a colorbar with more space
    cbar = fig.colorbar(im, ax=ax, orientation='horizontal', label='°C', shrink=0.7, pad=0.1)
    
    # Adjust the colorbar's position
    cbar.ax.yaxis.set_ticks_position('right')  # Put colorbar ticks on the right side

    # Save the plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"mean_temperature_{year}.png"), dpi=150)
    plt.close(fig)

print("✅ Mean temperature maps with adjusted colorbar spacing generated!")


### Monthly

In [ ]:
import xarray as xr
import pandas as pd
import os
import numpy as np

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\ERA5\Temperature\2conversion_year_1979_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store monthly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    # Open the dataset using memory-efficient lazy loading
    ds = xr.open_dataset(file_path, chunks={'valid_time': 500})
    
    # Extract year and month from the 'valid_time' dimension
    ds = ds.assign_coords(year=ds['valid_time'].dt.year, month=ds['valid_time'].dt.month)
    
    # Group by year and month and compute stats
    grouped = ds.groupby('month')

    for month in range(1, 13):
        if month not in grouped.groups:
            continue  # Skip if that month isn't in the file

        ds_month = grouped[month]
        temp_data = ds_month['t2m'].values

        # Compute stats over time (axis 0)
        mean = np.nanmean(temp_data, axis=0)
        median = np.nanmedian(temp_data, axis=0)
        std = np.nanstd(temp_data, axis=0)
        max_ = np.nanmax(temp_data, axis=0)
        min_ = np.nanmin(temp_data, axis=0)

        # Create a Dataset for the stats
        stats = xr.Dataset({
            'Mean': (['latitude', 'longitude'], mean),
            'Median': (['latitude', 'longitude'], median),
            'Std': (['latitude', 'longitude'], std),
            'Max': (['latitude', 'longitude'], max_),
            'Min': (['latitude', 'longitude'], min_)
        }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

        # Convert to DataFrame
        stats_df = stats.to_dataframe().reset_index()

        # Add time labels
        year_val = int(ds['year'].values[0])
        stats_df['Year'] = year_val
        stats_df['Month'] = month
        stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])

        df_list.append(stats_df)

# Combine all months from all years
df_final = pd.concat(df_list)

# Preview (optional)
print(df_final)


In [ ]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\ERA5\Temperature\monthly_temperature_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\ERA5\Temperature\monthly_temperature_stats.nc"
output_folder = r"E:\IPMA\ERA5\Temperature\maps_month"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Temperature (°C)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'coolwarm'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']

# Loop through all years and months
for year in ds['Year'].values:
    for month in ds['Month'].values:
        data = ds[var].sel(Year=year, Month=month)

        # Skip empty or all-NaN values
        if np.isnan(data).all():
            continue

        # Plotting
        fig = plt.figure(figsize=(7, 6))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'{title} - {year} / {month:02d}', fontsize=12)
        ax.coastlines(resolution='10m', linewidth=0.8)
        ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        # Gridlines (bottom and left only)
        gl = ax.gridlines(draw_labels=True, linewidth=0)
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = True
        gl.left_labels = True

        # Colorbar
        cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.7, pad=0.1)
        cbar.set_label('°C')
        cbar.ax.tick_params(labelsize=8)

        # Save figure
        plt.tight_layout()
        filename = f"mean_temperature_{year}{month:02d}.png"
        plt.savefig(os.path.join(output_folder, filename), dpi=150)
        plt.close(fig)

print("✅ Monthly temperature maps generated for all years and months!")


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\ERA5\Temperature\monthly_temperature_stats.nc"
output_folder = r"E:\IPMA\ERA5\Temperature\maps_month"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Temperature (°C)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'coolwarm'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']

# Loop through all years
for year in ds['Year'].values:
    fig, axs = plt.subplots(3, 4, figsize=(15, 10),
                            subplot_kw={'projection': ccrs.PlateCarree()})
    fig.suptitle(f'{title} - {year}', fontsize=16)

    for i, month in enumerate(ds['Month'].values):
        row, col = divmod(i, 4)
        ax = axs[row, col]

        # Extract data
        data = ds[var].sel(Year=year, Month=month)

        # Skip if no data
        if np.isnan(data).all():
            ax.set_visible(False)
            continue

        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'Month {month:02d}', fontsize=10)
        ax.coastlines(resolution='10m', linewidth=0.6)
        ax.add_feature(cfeature.BORDERS, linewidth=0.4)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        # Gridlines (bottom and left only)
        gl = ax.gridlines(draw_labels=True, linewidth=0)
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = row == 2
        gl.left_labels = col == 0

    # Add a single colorbar for all subplots
    cbar_ax = fig.add_axes([0.92, 0.25, 0.015, 0.5])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.set_label('°C')

    # Save figure
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    filename = f"mean_temperature_year_{year}.png"
    plt.savefig(os.path.join(output_folder, filename), dpi=150)
    plt.close(fig)

print("✅ Yearly temperature grids generated (3x4 months per year)!")


### Daily

In [ ]:
import xarray as xr
import pandas as pd
import os
import numpy as np

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\Temperature\2conversion_year_1979_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store daily DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    # Open the dataset using lazy loading (chunking for memory efficiency)
    ds = xr.open_dataset(file_path, chunks={'valid_time': 500})
    
    # Add year, month, and day as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month,
        day=ds['valid_time'].dt.day
    )
    
    # Group by each day
    grouped = ds.groupby('day')
    
    for day_val in np.unique(ds['day'].values):
        # Subset for this day
        ds_day = ds.sel(valid_time=ds['valid_time'].dt.day == day_val)
        
        if ds_day['valid_time'].size == 0:
            continue  # Skip empty days
        
        temp_data = ds_day['t2m'].values  # shape: (time_in_day, lat, lon)

        # Compute stats over the time dimension (axis 0 = hourly values of that day)
        mean = np.nanmean(temp_data, axis=0)
        median = np.nanmedian(temp_data, axis=0)
        std = np.nanstd(temp_data, axis=0)
        max_ = np.nanmax(temp_data, axis=0)
        min_ = np.nanmin(temp_data, axis=0)

        # Create a Dataset for the stats
        stats = xr.Dataset({
            'Mean': (['latitude', 'longitude'], mean),
            'Median': (['latitude', 'longitude'], median),
            'Std': (['latitude', 'longitude'], std),
            'Max': (['latitude', 'longitude'], max_),
            'Min': (['latitude', 'longitude'], min_)
        }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

        # Convert to DataFrame
        stats_df = stats.to_dataframe().reset_index()

        # Add time labels
        year_val = int(ds['year'].values[0])
        month_val = int(ds.sel(valid_time=ds['valid_time'].dt.day == day_val)['month'].values[0])
        stats_df['Year'] = year_val
        stats_df['Month'] = month_val
        stats_df['Day'] = int(day_val)
        
        # Set multi-index
        stats_df = stats_df.set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])

        df_list.append(stats_df)

# Combine all days from all years
df_final = pd.concat(df_list)


In [ ]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"D:\IPMA\ERA5\Temperature\daily_temperature_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")

## Current Way

### Yearly

In [ ]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\Temperature\1raw_year_1979_2024"
output_folder = r"D:\IPMA\ERA5\Temperature\yearly_temperature_stats"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")

    # Load dataset and chunk
    ds = xr.open_dataset(file_path)
    ds = ds.chunk({'valid_time': 500})
    
    # Ensure time coordinate exists
    if 'valid_time' not in ds:
        raise ValueError(f"'valid_time' not found in {file}")
    
    # Extract temperature variable
    temp_data = ds['t2m'].values  # shape: (time, lat, lon)

    # Compute yearly statistics
    mean = np.nanmean(temp_data, axis=0)
    median = np.nanmedian(temp_data, axis=0)
    std = np.nanstd(temp_data, axis=0)
    max_ = np.nanmax(temp_data, axis=0)
    min_ = np.nanmin(temp_data, axis=0)

    # Create xarray dataset with yearly stats
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

    # Add the year as a coordinate
    stats = stats.expand_dims({'Year': [year_val]})

    # Save yearly file
    output_file_path = os.path.join(output_folder, f"yearly_temperature_stats_{year_val}.nc")
    stats.to_netcdf(output_file_path)
    print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


### Monthly

In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\Temperature\1raw_year_1979_2024"
output_folder = r"D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")
    
    ds = xr.open_dataset(file_path)  
    ds = ds.chunk({'valid_time': 500})  
    
    # Add year and month as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month
    )
    
    # Temporary list for this year
    df_list = []
    
    # Loop over each unique month in this file
    for month_val in tqdm(np.unique(ds['month'].values), desc=f"Year {year_val}", leave=False):
        ds_month = ds.sel(valid_time=(ds['month'] == month_val))
        if ds_month['valid_time'].size == 0:
            continue

        temp_data = ds_month['t2m'].values

        # Compute monthly statistics (same logic as daily before)
        mean = np.nanmean(temp_data, axis=0)
        median = np.nanmedian(temp_data, axis=0)
        std = np.nanstd(temp_data, axis=0)
        max_ = np.nanmax(temp_data, axis=0)
        min_ = np.nanmin(temp_data, axis=0)
        total = np.nansum(temp_data, axis=0)

        # Create Dataset for this month's stats
        stats = xr.Dataset({
            'Mean': (['latitude', 'longitude'], mean),
            'Median': (['latitude', 'longitude'], median),
            'Std': (['latitude', 'longitude'], std),
            'Max': (['latitude', 'longitude'], max_),
            'Min': (['latitude', 'longitude'], min_)
        }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

        # Convert to DataFrame
        stats_df = stats.to_dataframe().reset_index()

        # Add time labels
        stats_df['Year'] = year_val
        stats_df['Month'] = int(month_val)

        # Set multi-index
        stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])
        df_list.append(stats_df)
    
    # Save this year's results
    if df_list:  # Only save if data exists
        df_final = pd.concat(df_list)
        df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
        df_final_xr = df_final_xr.to_xarray()
        
        output_file_path = os.path.join(output_folder, f"monthly_temperature_stats_{year_val}.nc")
        df_final_xr.to_netcdf(output_file_path)
        print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


Processing 22 files from 2003 to 2024...


Files processed:   0%|          | 0/22 [00:00<?, ?it/s]


📂 Processing year 2003...


Files processed:   5%|▍         | 1/22 [00:21<07:40, 21.91s/it]

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly\monthly_temperature_stats_2003.nc

📂 Processing year 2004...


...

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly\monthly_temperature_stats_2022.nc

📂 Processing year 2023...


Files processed:  95%|█████████▌| 21/22 [07:51<00:23, 23.81s/it]

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly\monthly_temperature_stats_2023.nc

📂 Processing year 2024...


Files processed: 100%|██████████| 22/22 [08:33<00:00, 23.34s/it]

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly\monthly_temperature_stats_2024.nc

🎉 All yearly files saved successfully!


### Daily

In [2]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\Temperature\2conversion_year_1979_2024"
output_folder = r"D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")
    
    ds = xr.open_dataset(file_path)  
    ds = ds.chunk({'valid_time': 500})  
    
    # Add year, month, day as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month,
        day=ds['valid_time'].dt.day
    )
    
    # Temporary list for this year
    df_list = []
    
    # Loop over each unique month in this file
    for month_val in np.unique(ds['month'].values):
        days_in_month = np.unique(ds['day'].values[ds['month'].values == month_val])
        
        # Loop over each day in the month with a progress bar
        for day_val in tqdm(days_in_month, desc=f"Month {month_val}", leave=False):
            ds_day = ds.sel(
                valid_time=(ds['month'] == month_val) & (ds['day'] == day_val)
            )
            if ds_day['valid_time'].size == 0:
                continue
            
            temp_data = ds_day['t2m'].values

            # Compute daily statistics
            mean = np.nanmean(temp_data, axis=0)
            median = np.nanmedian(temp_data, axis=0)
            std = np.nanstd(temp_data, axis=0)
            max_ = np.nanmax(temp_data, axis=0)
            min_ = np.nanmin(temp_data, axis=0)

            # Create Dataset for this day's stats
            stats = xr.Dataset({
                'Mean': (['latitude', 'longitude'], mean),
                'Median': (['latitude', 'longitude'], median),
                'Std': (['latitude', 'longitude'], std),
                'Max': (['latitude', 'longitude'], max_),
                'Min': (['latitude', 'longitude'], min_)
            }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

            # Convert to DataFrame
            stats_df = stats.to_dataframe().reset_index()

            # Add time labels
            stats_df['Year'] = year_val
            stats_df['Month'] = month_val
            stats_df['Day'] = int(day_val)

            # Set multi-index
            stats_df = stats_df.set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
            df_list.append(stats_df)
    
    # Save this year's results
    if df_list:  # Only save if data exists
        df_final = pd.concat(df_list)
        df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
        df_final_xr = df_final_xr.to_xarray()
        
        output_file_path = os.path.join(output_folder, f"daily_temperature_stats_{year_val}.nc")
        df_final_xr.to_netcdf(output_file_path)
        print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


Processing 22 files from 2003 to 2024...


Files processed:   0%|          | 0/22 [00:00<?, ?it/s]


📂 Processing year 2003...


Files processed:   5%|▍         | 1/22 [00:33<11:48, 33.76s/it]

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly\daily_temperature_stats_2003.nc

📂 Processing year 2004...


...

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly\daily_temperature_stats_2022.nc

📂 Processing year 2023...


Files processed:  95%|█████████▌| 21/22 [12:10<00:35, 35.61s/it]

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly\daily_temperature_stats_2023.nc

📂 Processing year 2024...


Files processed: 100%|██████████| 22/22 [12:42<00:00, 34.67s/it]

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly\daily_temperature_stats_2024.nc

🎉 All yearly files saved successfully!
